In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import os

In [6]:
#sl:satisfaction_level---False:MinMaxScaler;True:StandardScaler
#le:last_evaluation---False:MinMaxScaler;True:StandardScaler
#npr:number_project---False:MinMaxScaler;True:StandardScaler
#amh:average_monthly_hours--False:MinMaxScaler;True:StandardScaler
#tsc:time_spend_company--False:MinMaxScaler;True:StandardScaler
#wa:Work_accident--False:MinMaxScaler;True:StandardScaler
#pl5:promotion_last_5years--False:MinMaxScaler;True:StandardScaler
#dp:department--False:LabelEncoding;True:OneHotEncoding
#slr:salary--False:LabelEncoding;True:OneHotEncoding
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv("./data/HR.csv")

    #1、清洗数据
    df=df.dropna(subset=["satisfaction_level","last_evaluation"])
    df=df[df["satisfaction_level"]<=1][df["salary"]!="nme"]
    #2、得到标注
    label = df["left"]
    df = df.drop("left", axis=1)
    #3、特征选择
    #4、特征处理
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=["satisfaction_level","last_evaluation","number_project",\
                "average_monthly_hours","time_spend_company","Work_accident",\
                "promotion_last_5years"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=\
                MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df[column_lst[i]]=\
                StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
    scaler_lst=[slr,dp]
    column_lst=["salary","department"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=="salary":
                # low 为0,medium为1，high为2
                df[column_lst[i]]=[map_salary(s) for s in df["salary"].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])    
        df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df.values),label  
    return df,label            
                

def map_salary(s):
    d=dict([("low",0),("medium",1),("high",2)])
    return d.get(s,0)

# hr_preprocessing(lower_d=True,ld_n=5)

(array([[-0.14989291, -0.4220161 , -0.42173539,  0.12233499,  0.18495309],
        [-0.14643287,  0.41677176,  0.1984778 , -0.25833896, -0.14965431],
        [-0.26709532,  0.68671233,  0.18811401, -0.1507462 ,  0.58382736],
        ...,
        [-0.14803561, -0.43441955, -0.503113  ,  0.04173146,  0.19932448],
        [-0.29885501,  0.76297807, -0.24200472,  0.09424764,  0.44070635],
        [-0.15087015, -0.41003776, -0.49717793,  0.04236114,  0.20437636]]),
 0        1
 1        1
 2        1
 3        1
 4        1
 5        1
 6        1
 7        1
 8        1
 9        1
 10       1
 11       1
 12       1
 13       1
 14       1
 15       1
 16       1
 17       1
 18       1
 19       1
 20       1
 21       1
 22       1
 23       1
 24       1
 25       1
 26       1
 27       1
 28       1
 29       1
         ..
 14969    1
 14970    1
 14971    1
 14972    1
 14973    1
 14974    1
 14975    1
 14976    1
 14977    1
 14978    1
 14979    1
 14980    1
 14981    1
 14982 